In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import cv2
import numpy as np
import urllib.request
import os
import glob
import re
import socket

In [2]:
from time import sleep

In [3]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("http://image-net.org/")

In [4]:
busqueda = input('Ingrese su búsqueda: ')

Ingrese su búsqueda: cat


In [5]:
searchBar = driver.find_element_by_id("searchbox")

searchBar.send_keys(busqueda)

In [6]:
searchBar.send_keys(Keys.RETURN)

In [7]:
results = driver.find_elements_by_css_selector("a")
    
#For para obtener los links distintos de synset que arrojó la busqueda
links = []
for result in results:
    link = result.get_attribute("href")
    #Verifica que sea un link de synset
    if (re.search("http://image-net.org/synset?", link)):
        if (len(links)):
            #Si hay mas de un link, verifica que no sea igual al ultimo y lo guarda
            if (link != links[-1]):
                links.append(link)
        else:
            links.append(link)

In [8]:
imgsLinks = []
#For para acceder a todos los links de synset y guardar los links de las imagenes. 
for link in range(len(links)):
    try:
        #Pasos para para llegar a la ventana que tiene los links de las imagenes
        driver.get(links[link])
        sleep(4)
        downloads = driver.find_element_by_link_text('Downloads')
        downloads.click()
        downloadButton = driver.find_element_by_xpath('//*[@id="downloadstab"]/a[1]')
        downloadButton.click()
        
        #Guardar los links en la lista
        page = requests.get(driver.current_url)
        soup = BeautifulSoup(page.content, 'lxml')
        soup = soup.text
        imgsLinks.extend(soup.splitlines())
    except:
        print("No se pudo acceder a un link de synset: ", links[link])
driver.quit()

In [9]:
#Obtener los conjuntos de entrenamiento y pruebas. 
train_num = int(len(imgsLinks) * 0.80)
trainLinks = []
testLinks = []

for i in range(len(imgsLinks)):
    if i < train_num:
        trainLinks.append(imgsLinks[i])
    else:
        testLinks.append(imgsLinks[i])

In [10]:
newpath_train = r'train\\' + busqueda
newpath_test = r'test\\' + busqueda

In [11]:
#Crear los directorios o prepararlos

if not os.path.exists(newpath_train):
    os.makedirs(newpath_train)
else:
    #Si la carpeta ya existe borrar su contenido 
    r = glob.glob(newpath_train +'\*')
    for i in r:
        os.remove(i)
    
if not os.path.exists(newpath_test):
    os.makedirs(newpath_test)
else:
    #Si la carpeta ya existe borrar su contenido
    r = glob.glob(newpath_test +'\*')
    for i in r:
        os.remove(i)
    
name_train = newpath_train + "\image"
name_test = newpath_test + "\image"

socket.setdefaulttimeout(15)

In [12]:
count = 1
#Descargar las imagenes de entrenamiento 
for url in trainLinks:
    file_name = name_train + str(count) + ".jpg"
    count = count + 1
    try:
        urllib.request.urlretrieve(url, file_name)
    except Exception as e:
        print('Imagen no disponible: ', url)

Imagen no disponible:  http://static.flickr.com/121/287178609_e2da31d419.jpg
Imagen no disponible:  http://farm1.static.flickr.com/118/287178612_7f68f9563b.jpg
Imagen no disponible:  http://www.ualberta.ca/~dcarney/Pictures/Tigerlily1.jpg
Imagen no disponible:  http://www.mypage.tsn.cc/wildkatt/White%20Tiger%20-%20silver%20marble%205.jpg
Imagen no disponible:  http://tigertouch.org/cats/iriomote_cat.jpg
Imagen no disponible:  http://static.flickr.com/6/5733476_9266d6999b.jpg
Imagen no disponible:  http://farm1.static.flickr.com/7/11928401_f4dfb7d3b4.jpg
Imagen no disponible:  http://lynx.uio.no/lynx/catsgportal/cat-website/catfolk/pictures/tiger-04.jpg
Imagen no disponible:  http://farm1.static.flickr.com/197/464579958_acedd32444.jpg
Imagen no disponible:  http://farm2.static.flickr.com/1020/759839739_60c5a66df0.jpg
Imagen no disponible:  http://farm3.static.flickr.com/2409/2115133686_808ce0a5a5.jpg
Imagen no disponible:  http://farm1.static.flickr.com/185/426029368_3a9612f006.jpg
Imag

KeyboardInterrupt: 

In [ ]:
count = 1
#Descargar las imagenes de pruebas
for url in testLinks:
    file_name = name_test + str(count) + ".jpg"
    count = count + 1
    try:
        urllib.request.urlretrieve(url, file_name)
    except Exception as e:
        print('Imagen no disponible: ', url)